**Tabla de contenido**

- [Lectura de datos](#Lectura-de-datos)
- [Conclusión](#Conclusion)

En este cuaderno, determinaremos cuántos vehículos son necesarios para atender el incremento del 20% en Turbaco.

# Lectura de datos

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

import plotly.graph_objects as go
import plotly.express as px

project_root = next(p for p in Path.cwd().parents if (p / 'data').exists()) 
file_path = lambda file : os.path.join(project_root,'data',file)
data_entregas = pd.read_csv(file_path('DataSet_Entregas.csv'))

Obtengamos solo los datos para la terminal de origen Turbaco

In [2]:
data_entregas = data_entregas[data_entregas['Terminal_Origen']=='9. Turbaco']
data_entregas = data_entregas[['Terminal_Origen',
                               'Producto',
                               'Fecha_Recogida',
                               'Peso',
                               'Peso_Volumen',
                               'Unidades']]
data_entregas.head()

,Terminal_Origen,Producto,Fecha_Recogida,Peso,Peso_Volumen,Unidades
1,9. Turbaco,Paquetería,2023-12-29,1.0,1.00,1
18,9. Turbaco,Paquetería,2023-12-30,1.0,0.80,1
25,9. Turbaco,Mercancía XL,2023-12-29,126.0,114.03,1
30,9. Turbaco,Paquetería,2023-12-29,1.0,1.00,1
71,9. Turbaco,Paquetería,2023-12-29,1.0,0.00,1


Ordenemos este dataset por fecha.

In [3]:
data_entregas['Fecha_Recogida'] = pd.to_datetime(data_entregas['Fecha_Recogida'], format='%Y-%m-%d')
data_entregas = data_entregas.sort_values('Fecha_Recogida').reset_index(drop=True)
data_entregas.head(10)

,Terminal_Origen,Producto,Fecha_Recogida,Peso,Peso_Volumen,Unidades
0,9. Turbaco,Paquetería,2023-06-20,1.0,1.00,1
1,9. Turbaco,Paquetería,2023-07-18,1.0,0.70,1
2,9. Turbaco,Paquetería,2023-07-28,1.0,0.01,1
3,9. Turbaco,Paquetería,2023-08-29,1.0,1.00,1
4,9. Turbaco,Paquetería,2023-09-07,1.0,0.00,1
5,9. Turbaco,Paquetería,2023-09-18,1.0,0.72,1
6,9. Turbaco,Paquetería,2023-10-02,3.0,5.00,1
7,9. Turbaco,Paquetería,2023-10-11,1.0,0.01,1
8,9. Turbaco,Mercancía,2023-10-11,10.0,30.00,1
9,9. Turbaco,Mercancía,2023-10-18,11.0,16.64,1


Sumemos los productos que se envian por día.

In [4]:
df = data_entregas.groupby(by=['Fecha_Recogida']).agg(
    total_unidades =('Unidades','sum')
).reset_index()
df.head()

,Fecha_Recogida,total_unidades
0,2023-06-20,1
1,2023-07-18,1
2,2023-07-28,1
3,2023-08-29,1
4,2023-09-07,1


Calculemos el total de unidades recojidas a fin de mes.

In [5]:
unidades_por_mes = df.groupby(df['Fecha_Recogida'].dt.to_period('M').dt.end_time)['total_unidades'].sum().reset_index()
unidades_por_mes.columns = ['mes_fin', 'total_unidades_mes']
unidades_por_mes.head()

,mes_fin,total_unidades_mes
0,2023-06-30 23:59:59.999999999,1
1,2023-07-31 23:59:59.999999999,2
2,2023-08-31 23:59:59.999999999,1
3,2023-09-30 23:59:59.999999999,2
4,2023-10-31 23:59:59.999999999,8


Veamos la serie temporal de Turbaco.

In [6]:
def plot_time_series(df, x_var, y_var, title, xaxis_title, yaxis_title,dash='dash'):
    # Asegurar que la variable del eje x esté en formato datetime
    if not df[x_var].dtype.name.startswith('datetime'):
        df[x_var] = pd.to_datetime(df[x_var])

    # Crear figura
    fig = go.Figure()

    # Agregar la serie con línea punteada
    fig.add_trace(go.Scatter(
        x=df[x_var],
        y=df[y_var],
        mode='lines',
        name=y_var.replace('_', ' ').capitalize(),
        line=dict(color='blue', width=2, dash=dash)
    ))

    # Actualizar diseño
    fig.update_layout(
        title=f'<b>{title}</b>',
        title_font=dict(size=20, family='Arial', color='black'),
        xaxis_title=f'<b>{xaxis_title}</b>',
        yaxis_title=f'<b>{yaxis_title}</b>',
        font=dict(family='Arial', size=12, color='black'),
        plot_bgcolor='white',
        paper_bgcolor='white',
        hovermode='x unified',
        height=400,
        margin=dict(l=50, r=50, b=50, t=80)
    )

    # Formato año-mes en eje x
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', tickformat='%Y-%m')
    fig.update_yaxes(showgrid=True, gridcolor='lightgray')

    fig.show()
    return fig

In [7]:
fig =plot_time_series(df,
                       'Fecha_Recogida', 'total_unidades', 
                 title='',
                 xaxis_title='Fecha',
                 yaxis_title='Unidades',
                     dash='dash')

Ahora veamos la serie para el total mensual.

In [24]:
fig =plot_time_series(unidades_por_mes,
                       'mes_fin', 'total_unidades_mes', 
                 title='',
                 xaxis_title='Fecha',
                 yaxis_title='Unidades',
                     dash='dash')

Veamos la cantitad del  último mes.

In [14]:
unidades_por_mes.tail(1)

,mes_fin,total_unidades_mes
29,2025-11-30 23:59:59.999999999,4056


Perfecto, ahora carguemos la información de los vehiculos

In [9]:
vehiculos = pd.read_csv(file_path('Data_Set_Moviles.csv'))
vehiculos =vehiculos[vehiculos['Terminal']=='9. Turbaco']
vehiculos.head()

,Terminal,Capacidad,Unidad,Movil
926,9. Turbaco,23.0,MT3,1850
927,9. Turbaco,23.0,MT3,2134
928,9. Turbaco,13.0,MT3,2137
929,9. Turbaco,23.0,MT3,2241
930,9. Turbaco,23.0,MT3,2292


Ok, contemos cuántos vehículos hay por tipo capacidad.

In [10]:
coun_vehi = vehiculos['Capacidad'].value_counts().reset_index()
coun_vehi.columns = ['capacidad', 'cantidad']
coun_vehi.head(8)

,capacidad,cantidad
0,13.0,12
1,23.0,10
2,81.0,4
3,36.0,1
4,4.0,1
5,41.0,1
6,20.0,1


ok. Calculemos la capacidad total de la flota.

In [11]:
coun_vehi['capacidad total']=coun_vehi['capacidad']*coun_vehi['cantidad']
capacidad_total_flota = coun_vehi['capacidad total'].sum()
capacidad_total_flota

np.float64(811.0)

Perfecto, la flota total puede con 811 unidades. 

Veamos cuántos viajes realizó la flota en el último mes, suponiendo que todos los vehículos salen al mismo tiempo para satisfacer la demanda.

demanda último mes = 4056

In [15]:
viajes_ultimomes = 4056/capacidad_total_flota
print(f"para suplir la demanda la flota realizó {round(viajes_ultimomes)} viajes")

para suplir la demanda la flota realizó 5 viajes


Veamos cuantos viajes debe hacer la flota para suplir la nueva demanda.

In [16]:
unidades_prox_mes = 4056 + (4056* 0.2)
Viajesnecesarios = unidades_prox_mes/capacidad_total_flota
print(f"para suplir el incremento del 20% es necesario {round(Viajesnecesarios)} viajes")


para suplir el incremento del 20% es necesario 6 viajes


Esto quiere decir que si pueden aumentar de 5 a 6 viajes el próximo mes sin comprar nuevos carros, entonces no se necesitan vehículos adicionales.

Que pasa si si no se puede aumentar la cantidad de viajes?

In [17]:
capacidad_necesaria_por_viaje = (4056 + (4056* 0.2))/5
#calculemos la capacidad requerida por viaje
requerido_por_viaje = capacidad_necesaria_por_viaje - capacidad_total_flota
requerido_por_viaje

np.float64(162.43999999999994)

Perfecto, calculemos ahora si la cantidad de carros necesarios si no se pueden aumentar más viajes.

In [18]:
from itertools import combinations_with_replacement
# Capacidad extra necesaria
extra_necesario = requerido_por_viaje

# Tipos de carros disponibles (capacidad)
tipos_carros = coun_vehi['capacidad'].tolist()
# Ordenar de mayor a menor capacidad
tipos_carros.sort(reverse=True)

# Buscar la mejor combinación probando diferentes cantidades de carros
mejor_combinacion = None
menor_exceso = float('inf')
max_carros = 5  # Límite razonable de carros a añadir

for n in range(1, max_carros + 1):
    for comb in combinations_with_replacement(tipos_carros, n):
        capacidad_total = sum(comb)
        if capacidad_total >= extra_necesario:
            exceso = capacidad_total - extra_necesario
            if exceso < menor_exceso:
                menor_exceso = exceso
                mejor_combinacion = comb

# Resultados
print("Capacidad extra necesaria:", extra_necesario)
print("Mejor combinación encontrada:", mejor_combinacion)
print("Total carros añadidos:", len(mejor_combinacion))
print("Capacidad total añadida:", sum(mejor_combinacion))
print("Exceso:", menor_exceso)

Capacidad extra necesaria: 162.43999999999994
Mejor combinación encontrada: (81.0, 41.0, 41.0)
Total carros añadidos: 3
Capacidad total añadida: 163.0
Exceso: 0.5600000000000591


# Conclusion

Si los envíos se realizaran en 6 viajes mensuales, no sería necesario agregar más vehículos, ya que la capacidad actual de la flota puede absorber el incremento de demanda distribuido en ese número de viajes.

En caso de que solo sea posible realizar 5 viajes al mes, sería necesario incorporar tres carros adicionales: uno con capacidad de 81.0 y dos con capacidad de 41.0, para alcanzar la capacidad requerida en cada viaje.

Sin embargo, si la operación actual permite aumentar el número de viajes por mes, la flota existente sería suficiente y no se requerirían vehículos extras.
